# Aprenentatge profund per a imatges

A les lliçons anteriors, hem treballat extensament amb **dades tabulars**. Aquests conjunts de dades són generalment fàcils de representar i analitzar utilitzant eines com `pandas`, `scikit-learn` i diversos models estadístics.

Tanmateix, les **imatges** són fonamentalment diferents tant en estructura com en contingut. En lloc de files i columnes amb característiques etiquetades explícitament, una imatge és una graella de valors de píxels, normalment en 2D per a escala de grisos o en 3D per a color (alçada × amplada × canals). Per exemple, una imatge en color de mida 224×224 píxels amb 3 canals de color (RGB) conté més de 150.000 valors en brut, cap dels quals no està etiquetat amb característiques interpretables per humans com “edat” o “ingrés”.

Aquesta diferència condueix a diversos reptes importants:

- **Alta dimensionalitat**: Les imatges contenen moltes més característiques (píxels) que els conjunts de dades tabulars típics, cosa que augmenta la complexitat computacional i el risc de *overfitting*.
- **Estructura espacial**: Els píxels propers en una imatge sovint estan relacionats, formant vores, textures i patrons. Els models tabulars generalment no capten aquestes dependències locals.
- **Bretxa semàntica**: La relació entre els píxels en brut i els conceptes significatius (com un moic, una cara o un senyal de trànsit) és complexa i no lineal, i requereix models sofisticats per salvar aquesta bretxa.


En aquest mòdul explorarem com treballar amb dades d’imatges Per fer-ho, tornarem a utilitzar `PyTorch`.

## *Batches*

Quan entrenem models d’aprenentatge profund amb imatges, no alimentem tot el conjunt de dades alhora, com fèiem amb les dades tabulars. En lloc d’això, agrupem múltiples imatges en *batches*.

Un *batch* és una col·lecció de mostres (p. ex. imatges i les seves etiquetes) processades juntes en una sola passada endavant i enrere. Com a resultat les dades d'imatge són representades com un tensor de 4 dimensions:

```(batch_size, channels, height, width)
```

## El conjunt de dades MNIST

Per començar a treballar amb dades d’imatges en la pràctica, utilitzarem un dels conjunts de dades de referència més coneguts: **MNIST**. **MNIST** significa *Modified National Institute of Standards and Technology*. Consta de **70.000 imatges en escala de grisos** de xifres manuscrites (del 0 al 9), dividides en:

- **60.000 imatges d’entrenament**
- **10.000 imatges de prova**

Cada imatge és:

- **28 × 28 píxels**
- **En escala de grisos** (és a dir, un sol canal)
- **Etiquetada** amb la xifra correcta (0–9)

El conjunt de dades MNIST ha estat àmpliament utilitzat com a banc de proves per a algorismes d’aprenentatge automàtic i aprenentatge profund. Aquest conjunt de dades és ideal per aprendre perquè és petit, net i ja preprocesat, però alhora ofereix una complexitat realista en les xifres manuscrites.

### Exemple d’imatges MNIST

A continuació es mostra una mostra de xifres MNIST:

![MNIST Examples](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Cada fila a la imatge anterior mostra les xifres del 0 al 9 escrites per persones diferents. Com pots veure, algunes xifres s’escriuen de maneres molt diferents, i és per això que necessitem l’aprenentatge automàtic per reconèixer-les automàticament.

In [1]:
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm

## Carrega de dades

El processament de dades, com ja hem vist, és una part fonamental de qualsevol projecte d'aprenentatge profund. Per treballar amb dades de manera eficient, PyTorch ofereix dues classes: ``Dataset`` i ``DataLoader``.

El ``Dataset`` és una classe que representa un conjunt de dades. Serveix per carregar, transformar i accedir als elements individuals del conjunt. PyTorch inclou diversos datasets predefinits com MNIST, CIFAR-10 o ImageNet, però també podem crear els nostres propis datasets personalitzats heretant de torch.utils.data.Dataset i implementant els mètodes __len__() i __getitem__().

Un cop tenim el dataset, el ``DataLoader`` s’encarrega de gestionar la manera com aquestes dades s’entreguen al model. Permet dividir les dades en **batches**, barrejar-les (shuffle) i carregar-les en paral·lel utilitzant múltiples fils (*workers*).

In [2]:
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset_train = datasets.MNIST(root='data', train=True, download=True, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE)

dataset_val = datasets.MNIST(root='data', train=False, download=True, transform=transform)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE)

100%|██████████| 9.91M/9.91M [00:06<00:00, 1.47MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 215kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 241kB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


In [3]:
for img, gt in dataset_train:
	print(img.shape)
	print(gt)
	break

torch.Size([1, 28, 28])
5


In [4]:
for batch, gt in dataloader_train:
	print(batch.shape)
	print(gt.shape)
	break

torch.Size([64, 1, 28, 28])
torch.Size([64])


`nn.Sequential`` és una manera senzilla de construir una xarxa neuronal apilant les capes en ordre.

És útil quan el teu model és una cadena lineal de capes, sense ramificacions ni lògica personalitzada.

In [5]:
mlp_net = nn.Sequential(
    torch.nn.Linear(784, 10),
    nn.ReLU(),
    torch.nn.Linear(10, 10),
    torch.nn.Dropout(0.2),
    nn.ReLU(),
    torch.nn.Linear(10, 10)
)

### Entrenament

In [8]:
EPOCHS = 5
loss_fn = nn.CrossEntropyLoss()
LR = 1e-3
optimizer = torch.optim.Adam(mlp_net.parameters(), lr=LR)

In [10]:
running_loss = []
running_acc = []

running_test_loss = []
running_test_acc = []

for t in tqdm(range(EPOCHS), desc="Epochs"):
    batch_loss = 0
    batch_acc = 0

    i_batch = 0
    for i_batch, (x, y) in enumerate(dataloader_train):  # We have to iter the batches.
        mlp_net.train()
        x = x.reshape(x.shape[0], -1)  # Flatten images

        optimizer.zero_grad()
        y_pred = mlp_net(x)

        # 1. LOSS CALCULATION
        loss = loss_fn(y_pred, y)

        # 2. GRADIENT
        mlp_net.zero_grad()
        loss.backward()

        # 3. OPTIMISATION
        with torch.no_grad():
            optimizer.step()

        y_pred_labels = torch.argmax(y_pred, 1)
        batch_loss += loss.item()
        batch_acc += accuracy_score(y.cpu(), y_pred_labels.cpu())

    running_loss.append(batch_loss / (i_batch + 1))
    running_acc.append(batch_acc / (i_batch + 1))

        # 4. EVALUATION
    mlp_net.eval()  # Mode avaluació de la xarxa

    val_loss = 0
    val_acc = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            x_val = x_val.reshape(x_val.shape[0], -1)
            y_pred_val = mlp_net(x_val)
            val_loss += loss_fn(y_pred_val, y_val).item()
            y_pred_labels_val = torch.argmax(y_pred_val, 1)
            val_acc += accuracy_score(y_val.cpu(), y_pred_labels_val.cpu())

    running_test_loss.append(val_loss / len(dataloader_val))
    running_test_acc.append(val_acc / len(dataloader_val))

    print(f"Epoch {t+1}: Train Loss={running_loss[-1]:.4f}, Train Acc={running_acc[-1]:.4f}, "
          f"Val Loss={running_test_loss[-1]:.4f}, Val Acc={running_test_acc[-1]:.4f}")

Epochs:  20%|██        | 1/5 [00:10<00:40, 10.23s/it]

Epoch 1: Train Loss=0.4684, Train Acc=0.8534, Val Loss=0.3005, Val Acc=0.9105


Epochs:  40%|████      | 2/5 [00:20<00:30, 10.24s/it]

Epoch 2: Train Loss=0.4520, Train Acc=0.8577, Val Loss=0.2905, Val Acc=0.9139


Epochs:  60%|██████    | 3/5 [00:31<00:21, 10.59s/it]

Epoch 3: Train Loss=0.4377, Train Acc=0.8641, Val Loss=0.2899, Val Acc=0.9146


Epochs:  80%|████████  | 4/5 [00:43<00:11, 11.11s/it]

Epoch 4: Train Loss=0.4257, Train Acc=0.8674, Val Loss=0.2882, Val Acc=0.9163


Epochs: 100%|██████████| 5/5 [00:55<00:00, 11.08s/it]

Epoch 5: Train Loss=0.4213, Train Acc=0.8708, Val Loss=0.2757, Val Acc=0.9201


## Tasca a fer

1. Seleccionar la funció de pèrdua a emprar per poder entrenar el model.
2. Afegir la validació al bucle del `MNIST` i contestar a la pregunta de si hi ha *overfitting*?
2. Crear dos objectes `DataLoader` pel següent [dataset](https://github.com/miquelmn/aa_2526/releases/download/pr3/p4.tar.gz). Per fer-ho haureu de primer descarregar les imatges de l'enllaç, i després carregar les dades emprant [ImageFolder](https://docs.pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html) de `PyTorch`.
3. Entrenar un **MLP** amb `PyTorch` per tal d'identificar les classes. Prova primer amb una mida d'imatges més petita (64 per 64 píxels) fins a la mida original.

#### Creació del *DataLoader*

In [32]:
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001

# Transformació per les imatges
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Dataset d'entrenament
train_dataset = datasets.ImageFolder(root='datasetBirds/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

# Dataset de test
test_dataset = datasets.ImageFolder(root='datasetBirds/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

num_classes = len(train_dataset.classes)
input_size = 64 * 64 * 3  # Si les imatges són RGB

# Comprovar classes
print("Classes detectades:", train_dataset.classes)

Classes detectades: ['0', '4']


In [33]:
for img, gt in train_dataset:
	print(img.shape)
	print(gt)
	break

torch.Size([3, 64, 64])
0


In [34]:
for batch, gt in train_loader:
	print(batch.shape)
	print(gt.shape)
	break

torch.Size([32, 3, 64, 64])
torch.Size([32])


In [35]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten image
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

mlp_net = MLP(input_size=input_size, num_classes=num_classes)

In [36]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_net.parameters(), lr=LR)

In [37]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

running_loss = []
running_acc = []
running_val_loss = []
running_val_acc = []

for epoch in range(EPOCHS):
    # ---- ENTRENAMENT ----
    mlp_net.train()
    train_loss = 0
    train_acc = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        y_pred = mlp_net(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        y_pred_labels = torch.argmax(y_pred, dim=1)
        train_acc += accuracy_score(y.cpu(), y_pred_labels.cpu())

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    running_loss.append(train_loss)
    running_acc.append(train_acc)

    # ---- VALIDACIÓ ----
    mlp_net.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for x_val, y_val in test_loader:
            y_pred_val = mlp_net(x_val)
            val_loss += criterion(y_pred_val, y_val).item()
            y_pred_labels_val = torch.argmax(y_pred_val, dim=1)
            val_acc += accuracy_score(y_val.cpu(), y_pred_labels_val.cpu())

    val_loss /= len(test_loader)
    val_acc /= len(test_loader)

    running_val_loss.append(val_loss)
    running_val_acc.append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
          f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

Epoch 1/10: Train Loss=6.8810, Train Acc=0.8616, Val Loss=12.6818, Val Acc=0.5000
Epoch 2/10: Train Loss=2.7765, Train Acc=0.6384, Val Loss=4.1659, Val Acc=0.5000
Epoch 3/10: Train Loss=1.3735, Train Acc=0.4643, Val Loss=0.6946, Val Acc=0.5000
Epoch 4/10: Train Loss=0.7109, Train Acc=0.2262, Val Loss=0.6934, Val Acc=0.5000
Epoch 5/10: Train Loss=0.7016, Train Acc=0.2421, Val Loss=0.6933, Val Acc=0.5000
Epoch 6/10: Train Loss=0.6998, Train Acc=0.2421, Val Loss=0.6932, Val Acc=0.5000
Epoch 7/10: Train Loss=0.6989, Train Acc=0.2421, Val Loss=0.6932, Val Acc=0.5000
Epoch 8/10: Train Loss=0.6985, Train Acc=0.2579, Val Loss=0.6932, Val Acc=0.5000
Epoch 9/10: Train Loss=0.6983, Train Acc=0.2421, Val Loss=0.6932, Val Acc=0.5000
Epoch 10/10: Train Loss=0.6981, Train Acc=0.2421, Val Loss=0.6932, Val Acc=0.5000
